In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# cargar el conjunto de datos original
dataset_2017_name = 'datasets/GCI_Dataset_2007-2017.xlsx'
dataset_2017 = pd.read_excel(dataset_2017_name, index = False)

selected_year = 2017
init_year = 2017
final_year = 2018

In [ ]:
# filtrar para que los Attribute sean solamente Value
dataset_2017_only_values = dataset_2017[dataset_2017['Attribute'] == 'Value']
for year in dataset_2017_only_values['Edition'].unique():
    # separarlo en un conjunto por cada año
    tmp = dataset_2017_only_values[dataset_2017_only_values['Edition'] == year]
    tmp['Edition'] = tmp['Edition'].replace(year, year.split('-')[0])
    tmp.to_csv('datasets/GCI_' + year.split('-')[0] + '.csv', index = False)
    # se filtran las series que no son "bases"
    tmp = pd.read_csv('datasets/GCI_' + year.split('-')[0] + '.csv')
    tmp = tmp[tmp['Series code'].str.match('[0-9]')]
    tmp.to_csv('datasets/GCI_' + year.split('-')[0] + '.csv', index = False)


In [ ]:
# # reune el conjunto de atributos para los que Argentina no tiene ningún NaN en ningún año
# attributes = []
# for year in range(init_year, final_year):
#     tmp = pd.read_csv('datasets/GCI_' + str(year) + '.csv')
#     if len(attributes):
#         attributes = list(set(attributes) & set(tmp[~tmp['Argentina'].isna()]['Series Global ID'].to_list()))
    
#     else:
#         attributes = tmp[~tmp['Argentina'].isna()]['Series Global ID'].to_list()

# len(attributes)

In [ ]:
# # filtra todos los conjuntos de datos para que tengan los atributos para los que Argentina no tiene ningún NaN en ningún año
# for year in range(init_year, final_year):
#     tmp = pd.read_csv('datasets/GCI_' + str(year) + '.csv')
#     tmp = tmp[tmp['Series Global ID'].isin(attributes)]
#     tmp.to_csv('datasets/GCI_' + str(year) + '.csv', index = False)


In [ ]:
# para el año 2017, reune el conjunto de paises y atributos que al extraerlos 
# for selected_year in range(2007, 2018):
excluded_countries = []
excluded_attributes = []
tmp = pd.read_csv('datasets/GCI_' + str(selected_year) + '.csv')
tmp = tmp[tmp.columns.difference(['Dataset', 'Edition', 'Series Global ID', 'Series code', 'Series', 'Series unindented', 'Attribute'])].apply(pd.to_numeric, errors='coerce').isna()
while(tmp.sum(axis=0).max()):
    if (tmp.sum(axis=0).max() > tmp.sum(axis=1).max()):
        countries_nans = tmp.sum(axis=0)
        new_excluded_countries = countries_nans[countries_nans == countries_nans.max()].index.to_list()[0]
        excluded_countries.append(new_excluded_countries)
        tmp = tmp.drop(new_excluded_countries, axis=1)

    else:
        attributes_nans = tmp.sum(axis=1)
        new_excluded_attributes = attributes_nans[attributes_nans == attributes_nans.max()].index.to_list()[0]
        excluded_attributes.append(new_excluded_attributes)
        tmp = tmp.drop(new_excluded_attributes, axis=0)

excluded_countries = list(set(excluded_countries))
excluded_attributes = list(set(excluded_attributes))

df = pd.read_csv('datasets/GCI_' + str(selected_year) + '.csv')
df = df.drop(excluded_attributes, axis=0)
df = df.drop(excluded_countries, axis=1)

print(excluded_attributes, excluded_countries, tmp.shape[0], tmp.shape[1], tmp.shape[0] * tmp.shape[1])
print('---')

In [ ]:
# para todos los años, saca los atributos descubiertos en el paso anterior
for year in range(init_year, final_year):
    tmp = pd.read_csv('datasets/GCI_' + str(year) + '.csv')
    tmp = tmp.drop(excluded_attributes, axis=0)
    tmp.to_csv('datasets/GCI_' + str(year) + '.csv', index = False)
    

In [ ]:
# para todos los años, saca los países que tienen NaN
for year in range(init_year, final_year):
    tmp = pd.read_csv('datasets/GCI_' + str(year) + '.csv')
    new_columns = ['country'] + tmp['Series unindented'].to_list()
    tmp = tmp.T.reset_index()
    tmp.columns = new_columns
    tmp = tmp[~tmp['country'].isin(['Dataset', 'Edition', 'Series Global ID', 'Series code', 'Series', 'Series unindented', 'Attribute'])]
    tmp.to_csv('datasets/GCI_' + str(year) + '.csv', index = False)


In [ ]:
# trasponer el conjunto de datos, reindex, y retirar las filas que no son paises
for year in range(init_year, final_year):
    tmp = pd.read_csv('datasets/GCI_' + str(year) + '.csv')
    tmp = tmp[~(tmp.isna().sum(axis=1) > 0)]
    tmp.to_csv('datasets/GCI_' + str(year) + '.csv', index = False)


In [ ]:
# replacement = {
#     'Wastefulness of government spending, 1-7 (best)': 'Efficiency of government spending, 1-7 (best)',
#     'Unnamed: 2': 'Intellectual property protection, 1-7 (best)'
# }

# for year in range(init_year, final_year):
#     tmp = pd.read_csv('datasets/GCI_' + str(year) + '.csv')
#     tmp.rename(columns=replacement, inplace=True)
#     del tmp['Fixed broadband Internet subscriptions/100 pop.*']
#     tmp.to_csv('datasets/GCI_' + str(year) + '.csv', index = False)


In [ ]:
# # los guarda como Excel e imprime las dimensiones y la cantidad de NaNs
# for year in range(init_year, final_year):
#     tmp = pd.read_csv('datasets/GCI_' + str(year) + '.csv')
#     tmp = ~tmp.applymap(np.isreal)
#     tmp['country'] = False
#     print(tmp.shape)
#     print(tmp.sum().sum())
#     tmp.to_excel('datasets/GCI_' + str(year) + '.xlsx', index=False)
    

In [ ]:
# dataset_2019_name = 'datasets/WEF_GCI_4.0_2019_Dataset.xlsx'
# dataset_2019 = pd.read_excel(dataset_2019_name, index = False)

# tmp = dataset_2019[dataset_2019['Attribute'] == 'VALUE']
# tmp = tmp[tmp['Edition'] == '2017 backcast']
# tmp['ARG'].apply(pd.to_numeric, errors='coerce').to_list()

# pd.read_csv('datasets/GCI_2017.csv')['Argentina'].to_list()